In [ ]:
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import os
import posixpath

import torch
from torchvision.transforms import v2
import torchvision

import numpy as np
import cv2 as cv
import math
from scipy import interpolate
from numpy import asarray
import torchvision.transforms as transforms


In [ ]:
source1 = sorted(glob.glob('./output/txt/*.txt'), key = lambda i: (len(posixpath.basename(i)), i))

source2 = sorted(glob.glob('./output/afterunet/*_prediction.jpg'), key = lambda i: (len(posixpath.basename(i)), i))
source3 = sorted(glob.glob('./datasample/test_set_yolo/test*.jpg'), key = lambda i: (len(posixpath.basename(i)), i))
print(source1)
print(source2)
print(source3)

In [ ]:
def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, bottom))
    return result

# Post Processing
Brings Unet predictions back to original size

In [ ]:
from pathlib import Path
Path("./output/processed/").mkdir(parents=True, exist_ok=True)

destination_folder = './output/processed/'

for prediction, txt, image in zip(source2, source1, source3):
    
    with open(txt, 'r') as txts:
      content = txts.readline().strip()  
    
    numbers_as_string = content.split()
    numbers = [int(number) for number in numbers_as_string]
    w = numbers[0]
    h = numbers[1]
    x1 = numbers[2]
    y1 = numbers[3]
    x2 = numbers[4]
    y2 = numbers[5]
    img = Image.open(prediction)
    resize = v2.Resize(size=(512, 512))
    im1 = resize(img)
    resize2 = v2.Resize(size=(1024, 1024))
    im2 = resize2(im1)
    im_new1 = add_margin(im2, 0, -256, 0, -256, (0,))
    resize3 = v2.Resize(size=(h, w))
    im_new2 = resize3(im_new1)
    im_pad = add_margin(im_new2, y2, x2, y1, x1, (0,))

    base, ext = os.path.splitext(os.path.basename(prediction))  
    ext = ext.lower()  

    original_image = cv.imread(image)

    new_path = os.path.join(destination_folder, f"{base}{ext}")

    im_pad.save(new_path)
